In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
import seaborn as sns
from matplotlib import cm
import copy

## Load file folder

In [ ]:
def get_filelist(directory):

    filelist = []

    for home,dirs,files in os.walk(directory):

        for file in files:
            filelist.append(os.path.join(home, file))

    return filelist

## Get image list and image name

In [ ]:
def get_imagelist(directory):
    filelist = []
    imgList = []
    imgName = []

    for home,dirs,files in os.walk(directory):

        for file in files:
            filelist.append(os.path.join(home, file))
            imgName.append(file)
    for i in range(len(filelist)):
        filetype = os.path.splitext(filelist[i])[-1]
        if filetype == '.txt':
            if i == 0:
                temp = np.zeros((200,200))
            else:
                temp = np.zeros(np.shape(imgList[0]))
            txtfile = np.loadtxt(filelist[i]).astype(np.int64)
            for nx in range(np.shape(txtfile)[0]):
                for ny in range(np.shape(txtfile)[1]):
                    temp[nx][ny] =int(txtfile[nx][ny])
            imgList.append(temp)
        else:
            imgList.append(np.array(io.imread(filelist[i])))
    return imgList, imgName

## Load strain map folders and get image lists

In [ ]:
data_folder = './data_folder'
Folderlist = []
Foldername = []

for home,dirs,files in os.walk(data_folder):
    for dir in dirs:
        Folderlist.append(os.path.join(home, dir))
        Foldername.append(dir)
print(Folderlist)
print(Foldername)

num_samples = len(Folderlist)
print(num_samples)

## Rocking curve angle of every LiCoO2 particles

In [ ]:
two_theta_list = [
    np.linspace(10.1,12.1,21),
    np.linspace(-27.4,-23.5,40),
    np.linspace(-23.8,-19.8,41),
    np.linspace(-27,-23,41),    
    np.linspace(-39.7,-37.7,21),
    np.linspace(-4.7,-4.26,23),
    np.linspace(-7.5,-5.5,21),
    np.linspace(-22.55,-19.25,34)
]

## Plot rocking curve of every LiCoO2 particles

In [ ]:
fig,axs = plt.subplots(nrows=2, ncols=4,figsize=(24, 6),sharex=True)

for i in range(num_samples):
    imgList, imgName = get_imagelist(Folderlist[i])
    dim_z, dim_x, dim_y = imgList[1].shape
    profile_list = []
    
    for z in range(dim_z):
        profile_list.append(np.sum(imgList[1][z-1][np.nonzero(imgList[0]==1)])/np.count_nonzero(imgList[0]==1))
        
    axs[i//4,i%4].plot(two_theta_list[i], profile_list)
    axs[i//4,i%4].set_yticks([])
    
plt.show()

## Mean and standard deviation of d-spacing and z-bending strain maps

In [ ]:
d_spacing_mean = []
d_spacing_std = []
z_bending_mean = []
z_bending_std = []

for i in range(num_samples):
    imgList, imgName = get_imagelist(Folderlist[i])
    
    d_spacing_mean.append(np.sum(imgList[2][np.nonzero(imgList[0]==1)])/np.count_nonzero(imgList[0]==1))
    d_spacing_std.append(np.std(imgList[2][np.nonzero(imgList[0]==1)]))
    
    z_bending_mean.append(np.sum(imgList[3][np.nonzero(imgList[0]==1)])/np.count_nonzero(imgList[0]==1))
    z_bending_std.append(np.std(imgList[3][np.nonzero(imgList[0]==1)]))

## Calculate bias value of d-spacing and z-bending, and plot jointplot

In [ ]:
bias_d_spacing = d_spacing_mean
bias_z_bending = z_bending_mean
figzd, axeszd = plt.subplots(2, 4,figsize=(24,12),sharex=True,sharey=True)

for i in range(num_samples):
    imgList, imgName = get_imagelist(Folderlist[i])
    
    cmap = copy.copy(cm.get_cmap("RdBu_r"))

    dmin, dmax = -0.0015, 0.0015
    zmin, zmax = -0.18, 0.18
  
    sns.set_style("white")
    sns.jointplot(x=(imgList[3][np.nonzero(imgList[0] != 0)].flatten().T - bias_z_bending[i]) / 0.18, y=(imgList[2][np.nonzero(imgList[0] != 0)].flatten().T - bias_d_spacing[i]) / 0.0015, kind="kde", color='k', cmap=cmap, xlim=(-1, 1), ylim=(-1, 1), ax=axeszd[i//4, i%4])
    
    axeszd[i//4, i%4].set_xlim([-1, 1])
    axeszd[i//4, i%4].set_ylim([-1, 1])
    axeszd[i//4, i%4].set_xticks([-1, 0, 1])
    axeszd[i//4, i%4].set_yticks([-1, 0, 1])
    
plt.tight_layout()
plt.show()